# Imports and setup

### Auto-re-import python modules, useful for editing local fils

In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import itertools

import matplotlib as mpl
import pandas as pd
import seaborn as sns

mpl.rcParams["figure.max_open_warning"] = 0

# Handwritten local modules
import process_scop_sourmash_multisearch

# Read in data

In [3]:
analysis_outdir = (
    "s3://seanome-kmerseek/scope-benchmark/analysis-outputs/2024-10-09__protein_k5-20"
)

In [4]:
pipeline_outdir = (
    "s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-08__protein_k5-20"
)
! aws s3 ls $pipeline_outdir/

                           PRE multiqc/
                           PRE pipeline_info/
                           PRE seqkit/
                           PRE sourmash/


In [5]:
! aws s3 ls $pipeline_outdir/sourmash/

                           PRE multisearch/
                           PRE sigs/


In [6]:
! aws s3 ls --human-readable $pipeline_outdir/sourmash/multisearch/

2024-10-09 13:29:03    0 Bytes 
2024-10-09 03:28:12    6.0 MiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.protein.10.multisearch.csv
2024-10-09 06:17:15    5.7 MiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.protein.11.multisearch.csv
2024-10-09 04:22:24    5.5 MiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.protein.12.multisearch.csv
2024-10-09 08:07:15    5.5 MiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.protein.13.multisearch.csv
2024-10-09 07:10:35    5.4 MiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.protein.14.multisearch.csv
2024-10-09 12:35:34    5.4 MiB astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_00

In [7]:
# def make_multisearch_csv(
#     ksize,
#     outdir,
#     moltype,
#     query="astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa",
#     against="astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa",
# ):
#     basename = f"{query}--in--{against}.{moltype}.{ksize}.multisearch.csv"
#     csv = f"{outdir}/sourmash/multisearch/{basename}"
#     return csv


# scop_fixed = pd.read_csv(
#     "https://raw.githubusercontent.com/steineggerlab/foldseek-analysis/refs/heads/main/scopbenchmark/data/scop_lookup.fix.tsv",
#     sep="\t",
#     header=None,
#     names=["scop_id", "scop_class"],
#     index_col=0,
# ).squeeze()
# print(scop_fixed.shape)
# scop_fixed.head()

# scop_fixed.to_csv(
#     "s3://seanome-kmerseek/scope-benchmark/steineggerlab_foldseek-analysis_scop_fixed.csv"
# )

In [8]:
# # Skip ksizes 5 and 6 for now because those files are enormous, 2.3 GiB for k=5 and 175 MiB for k=6
# # -> Figure out how to use polars later
# ksizes = range(7, 21)

# for ksize in ksizes:
#     print(f"\n\n--- ksize: {ksize} --")
#     csv = make_multisearch_csv(ksize, pipeline_outdir, "protein")
#     print(f"\nReading {csv} ...")
#     %time multisearch = pd.read_csv(csv)
#     print("\tDone")

#     query_metadata = scop_utils.extract_scop_info_from_name(
#         multisearch.query_name, scop_fixed, "query", verbose=False
#     )

#     match_metadata = scop_utils.extract_scop_info_from_name(
#         multisearch.match_name, scop_fixed, "match", verbose=False
#     )

#     multisearch_metadata = multisearch.join(query_metadata, on="query_name").join(
#         match_metadata, on="match_name"
#     )

#     pq = f"{analysis_outdir}/00_cleaned_multisearch_results/scope40.multisearch.protein.k{ksize}.pq"
#     print(f"\nWriting {pq} ...")
#     %time multisearch_metadata.to_parquet(pq)
#     print(f"\tDone.")

In [9]:
analysis_outdir

's3://seanome-kmerseek/scope-benchmark/analysis-outputs/2024-10-09__protein_k5-20'

In [ ]:
# Skip ksizes 5 and 6 for now because those files are enormous, 2.3 GiB for k=5 and 175 MiB for k=6
# # -> Figure out how to use polars later
# ksizes = range(7, 21)
# moltype = "protein"

moltype_info = {
    # "protein": dict(
    #     ksizes=range(5, 21),
    #     pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-08__protein_k5-20",
    #     analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/2024-10-09__protein_k5-20",
    # ),
    # "dayhoff": dict(
    #     ksizes=range(5, 21),
    #     pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__dayhoff_k5-20",
    #     analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/2024-10-09__dayhoff_k5-20",
    # ),
    "hp": dict(
        ksizes=range(10, 20),
        pipeline_outdir="s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60",
        analysis_outdir="s3://seanome-kmerseek/scope-benchmark/analysis-outputs/2024-10-09__hp_k20-60",
    ),
}

for moltype, info in moltype_info.items():
    ksizes = info["ksizes"]
    analysis_outdir = info["analysis_outdir"]
    pipeline_outdir = info["pipeline_outdir"]
    for ksize in ksizes:
        try:
            multisearch_metadata_filtered = (
                process_scop_sourmash_multisearch.process_multisearch_scop_results(
                    pipeline_outdir, moltype, ksize, analysis_outdir, verbose=True
                )
            )
        except FileNotFoundError:
            pass
    # break
multisearch_metadata_filtered.head()



--- ksize: 10 --

Reading s3://seanome-kmerseek/scope-benchmark/pipeline-outputs/2024-10-09__hp_k20-60/sourmash/multisearch/astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa--in--astral-scopedom-seqres-gd-sel-gs-bib-40-2.08.part_001.fa.hp.10.multisearch.csv ...


In [ ]:
multisearch_metadata_filtered.dtypes